In [1]:
import pandas as pd
import geopandas as gpd
import arcpy
import os
import shutil
import math
import inspect
from yelpapi import YelpAPI
import pprint

yelp_api = YelpAPI("1TUPhIwjhhaG_ran5ng7KTMqHkYOA1tlIA8PUznpxrd2_Xojx1I0w8wGP7u_PRfDmyWKtHIUA8ScT-AEDidmQ19SSnfVz0ps94jlEvFVi-aiXMF__nz1eCIIrD-uXXYx")
src_file_path = inspect.getfile(lambda: None)
THIS_FOLDER = os.path.dirname(os.path.abspath(src_file_path))

# Summary

This notebook creates the land-use layer. It creates a golf courses layer using a combination of the Yelp API and the SBC Parcels layer. It also preprocesses the land-use categories for use in the EEMS modeling process. These will likely be updated and added to as the modeling process moves forward.

**Input Data Layers:**
- [SBC Parcels](https://databasin.org/datasets/008da0ba3dc14be18aa3bcfd90dc9615/)
- [SBC Agriculture](https://databasin.org/datasets/85876647b469495dbbde075274aaace1/)
- [SBC Lakes](https://databasin.org/datasets/0278b9eedb6a4536b5cbb155956ee284/)
- [SBC Desert](https://sbcblueprint.databasin.org/datasets/0b69c5ff46aa4f41820c2780d115c74e/)

**The YelpAPI provides list of golf course points for Santa Barbara County, which were intersected with SBC Parcels to get golf courses in SBC. The process is not completely automated and does require you to double check that the correct parcels are indeed selected. This was QA/QC'd in ArcGIS with a satellite imagery basemap. While this notebook shows how to use the Yelp API, it ultimately uses the QA/QC'd golf course layer**

In [10]:
response = yelp_api.search_query(categories='golf', location='Santa Barbara County, California', sort_by='rating', limit=50)

names = []
lats = []
lons = []

for v in response['businesses']:
    lat = v['coordinates']['latitude']
    lon = v['coordinates']['longitude']
    name = v['name']
    names.append(name)
    lats.append(lat)
    lons.append(lon)

data = {'name': names,
       'latitude': lats,
       'longitude': lons} 

df = pd.DataFrame(data)
# df.to_csv("out_data/lu/golf.csv", index=False)

**Load the initial layers**

In [11]:
arcpy.ResetEnvironments()
gdb_name = "Land_Use.gdb"
arcpy.env.overwriteOutput = True
path = os.path.join(THIS_FOLDER, gdb_name)
arcpy.CreateFileGDB_management("./", gdb_name)
arcpy.env.workspace = path
arcpy.env.scratchWorkspace = path

In [12]:
crs = arcpy.SpatialReference('NAD 1983 StatePlane California V FIPS 0405 (US Feet)')

in_data = ["SBC_Parcels", "SBC_Ag", "SBC_Lakes", "SBC_Desert", "SBC_Golf"]

for sf in in_data:
    path = os.path.join(THIS_FOLDER, "in_data/{}.shp".format(sf))
    arcpy.CopyFeatures_management(path, sf)
    arcpy.Project_management(sf, "{}_rpj".format(sf), crs)

**We are only interested in large lakes (<10 acres)**

In [13]:
selection = arcpy.management.SelectLayerByAttribute("SBC_Lakes_rpj", "NEW_SELECTION", "Shape_Area > 435600", None)
arcpy.CopyFeatures_management(selection, "Lg_Lakes")

<Result 'C:\\Users\\zrcan\\Documents\\GitHub\\RPP-SBC-EEMS\\Land_Use.gdb\\Lg_Lakes'>

**Now we merge the land-use categories together and set the fields accordingly**

In [17]:
lu_cats = ["SBC_Ag_rpj", "Lg_Lakes", "SBC_Desert_rpj", "SBC_Golf_rpj"]

arcpy.Merge_management(lu_cats, "LU_merged")

In [18]:
arcpy.AddField_management("LU_merged", "golf", "LONG")
arcpy.AddField_management("LU_merged", "ag", "LONG")
arcpy.AddField_management("LU_merged", "lake", "LONG")
arcpy.AddField_management("LU_merged", "desert", "LONG")

codeblock = """
def getClass(a):
    if a is None:
        return 0
    else:
        return 1
"""

arcpy.CalculateField_management("LU_merged",  "golf", "getClass(!apn!)", "PYTHON3", codeblock)
arcpy.CalculateField_management("LU_merged",  "ag", "getClass(!CropCatego!)", "PYTHON3", codeblock)
arcpy.CalculateField_management("LU_merged",  "lake", "getClass(!WETLAND_TY!)", "PYTHON3", codeblock)
arcpy.CalculateField_management("LU_merged",  "desert", "getClass(!WHRTYPE!)", "PYTHON3", codeblock)

FCfields = [f.name for f in arcpy.ListFields("LU_merged")]
DontDeleteFields = ['Shape_Length', 'Shape_Area', 'OBJECTID', 'OBJECTID_1', 'Shape', "golf", "ag", "lake", "desert"]
fields2Delete = list(set(FCfields) - set(DontDeleteFields))
arcpy.DeleteField_management("LU_merged", fields2Delete)

<Result 'C:\\Users\\zrcan\\Documents\\GitHub\\RPP-SBC-EEMS\\Land_Use.gdb\\LU_merged'>

**Create a single field to specify land-use category**

In [23]:
arcpy.AddField_management("LU_merged", "LU_Cat", "LONG")

expression = "getClass(!golf!, !ag!, !lake!, !desert!)"

codeblock = """
def getClass(a, d, l, g):
    if a == 1:
        return 1
    elif d == 1:
        return 2
    elif l == 1:
        return 3
    elif g == 1:
        return 4
    else:
        return 0"""

arcpy.CalculateField_management("LU_merged",  "LU_Cat", expression, "PYTHON3", codeblock)

<Result 'C:\\Users\\zrcan\\Documents\\GitHub\\RPP-SBC-EEMS\\Land_Use.gdb\\LU_merged'>

**Export the land-use category to out_data**

In [24]:
dir_path = os.path.join(THIS_FOLDER, "out_data/lu")
shutil.rmtree(dir_path)
os.mkdir(dir_path)

arcpy.conversion.FeatureClassToShapefile("LU_merged", 
                                        os.path.join(THIS_FOLDER, "out_data/lu"))

<Result 'C:\\Users\\zrcan\\Documents\\GitHub\\RPP-SBC-EEMS\\out_data\\lu'>